In [1]:
import pandas as pd
from scipy.spatial import distance_matrix

## Загрузка данных:

In [2]:
# Грузим датасет с данными о распространении вируса
df_covid = pd.read_csv('Regions.csv', sep=';')
df_covid.columns = ['date', 'reg', 'conf', 'recov', 'deaths', 'deaths_day', 'conf_day', 'recov_day']

# Меняем названия регионов которые не сходятся с названиями в другом датасете
df_covid = df_covid.replace(['Кабардино-Балкария', 'Карачаево-Черкессия', 'Саха (Якутия)', 'Северная Осетия', 'Удмуртия', 'ХМАО – Югра', 'Чечня', 'Чувашия'],\
           ['Кабардино-Балкарская', 'Карачаево-Черкесская', 'Саха /Якутия/', 'Северная Осетия - Алания', 'Удмуртская', 'Ханты-Мансийский Автономный округ - Югра', 'Чеченская', 'Чувашская Республика'])

# Обрезаем "край", "обл." и "АО"
df_covid.reg = df_covid.reg.apply(lambda x: x.split('край',1)[0].split('обл.', 1)[0].split('АО', 1)[0].rstrip())
df_covid

,date,reg,conf,recov,deaths,deaths_day,conf_day,recov_day
0,02.03.2020,Московская,1,0,0,0,1,0
1,03.03.2020,Московская,1,0,0,0,0,0
2,04.03.2020,Московская,1,0,0,0,0,0
3,05.03.2020,Московская,1,0,0,0,0,0
4,06.03.2020,Москва,5,0,0,0,5,0
...,...,...,...,...,...,...,...,...
6903,31.05.2020,Чеченская,1227,828,13,0,18,3
6904,31.05.2020,Чувашская Республика,2885,1242,14,0,87,30
6905,31.05.2020,Чукотский,80,54,1,0,2,1
6906,31.05.2020,Ямало-Ненецкий,2375,975,9,0,65,22


In [3]:
# Грузим датасет с геоданными
df_cities = pd.read_csv('tableconvert_csv_7t1xix.csv')
df_cities = df_cities[['Тип региона', 'Регион', 'Город', 'Широта', 'Долгота', 'Федеральный округ', 'Население']]
df_cities.columns = ['reg_type', 'reg', 'city', 'lat', 'long', 'fed_reg', 'population']

In [4]:
# Список региональных центров (столиц)
reg_centres = ['Красногорск', 'Всеволожск', 'Майкоп', 'Горно-Алтайск', 'Барнаул', 'Архангельск', 'Астрахань', 'Уфа', 'Белгород', 'Брянск', 'Улан-Удэ', 'Владимир', 'Волгоград', 'Вологда', 'Воронеж', 'Махачкала', 'Биробиджан', 'Чита', 'Иваново', 'Магас', 'Иркутск', 'Нальчик', 'Калининград', 'Элиста', 'Калуга', 'Петропавловск-Камчатский', 'Черкесск', 'Петрозаводск', 'Кемерово', 'Сыктывкар', 'Кострома', 'Краснодар', 'Красноярск', 'Симферополь', 'Курган', 'Курск', 'Санкт-Петербург', 'Липецк', 'Магадан', 'Йошкар-Ола', 'Саранск', 'Москва', 'Мурманск', 'Нарьян-Мар', 'Нижний Новгород', 'Великий Новгород', 'Новосибирск', 'Омск', 'Оренбург', 'Орёл', 'Пенза', 'Пермь', 'Владивосток', 'Псков', 'Ростов-на-Дону', 'Рязань', 'Самара', 'Санкт-Петербург', 'Саратов', 'Якутск', 'Южно-Сахалинск', 'Екатеринбург', 'Севастополь', 'Владикавказ', 'Смоленск', 'Ставрополь', 'Тамбов', 'Казань', 'Тверь', 'Томск', 'Тула', 'Кызыл', 'Тюмень', 'Ижевск', 'Ульяновск', 'Хабаровск', 'Абакан', 'Ханты-Мансийск', 'Челябинск', 'Грозный', 'Чебоксары', 'Анадырь', 'Салехард', 'Ярославль']

In [5]:
# Маркируем столицы субъектов единичкой в столбце is_centre
df_cities['is_centre'] = df_cities['city'].apply(lambda x: 1 if x in reg_centres else 0)

# Севастополь, Москву и Питер хардкодим
big_cities_indexes = df_cities[df_cities['city'].isna() & ((df_cities['reg'] == 'Севастополь') | \
                                                           (df_cities['reg'] == 'Москва') | \
                                                           (df_cities['reg'] == 'Санкт-Петербург'))].index
df_cities.loc[big_cities_indexes, 'is_centre'] = 1

df_cities.loc[df_cities[df_cities['reg'] == 'Севастополь'].index, 'city'] = 'Севастополь'
df_cities.loc[df_cities[df_cities['reg'] == 'Москва'].index, 'city'] = 'Москва'
df_cities.loc[df_cities[df_cities['reg'] == 'Санкт-Петербург'].index, 'city'] = 'Санкт-Петербург'

# Здесь, в силу того что есть два г. Киров и два г. Благовещенск в разных субъектах, тоже приходится хардкодить
small_cities_indexes = df_cities[((df_cities['city'] == 'Киров') & (df_cities['reg'] == 'Кировская')) |\
                                 ((df_cities['city'] == 'Благовещенск') & (df_cities['reg'] == 'Амурская'))].index
df_cities.loc[small_cities_indexes, 'is_centre'] = 1

# И с Чечней и Адлером какой то баг приходится поправить:
df_cities.loc[df_cities[df_cities['city'].isna() & (df_cities['reg'] == 'Чеченская')].index, 'city'] = 'Урус-Мартан'
adler = pd.DataFrame([['край', 'Краснодарский', 'Адлер', 43.438166, 39.911178, 'Южный', 111600, 0]])
adler.columns = df_cities.columns
adler.index = [len(df_cities)]
df_cities = df_cities.append(adler)

# Отдельный датасет только со столицами субъектов
df_cities_centres = df_cities[df_cities.is_centre == 1]
df_cities_centres.index = range(len(df_cities_centres))

## Список всех городов:

In [6]:
df_cities

,reg_type,reg,city,lat,long,fed_reg,population,is_centre
0,Респ,Адыгея,Адыгейск,44.878372,39.190172,Южный,12689,0
1,Респ,Адыгея,Майкоп,44.609827,40.100653,Южный,144055,1
2,Респ,Алтай,Горно-Алтайск,51.958268,85.960296,Сибирский,62861,1
3,край,Алтайский,Алейск,52.492091,82.779415,Сибирский,28528,0
4,край,Алтайский,Барнаул,53.348115,83.779836,Сибирский,635585,1
...,...,...,...,...,...,...,...,...
1108,обл,Ярославская,Рыбинск,58.048380,38.858338,Центральный,200771,0
1109,обл,Ярославская,Тутаев,57.867424,39.536823,Центральный,41001,0
1110,обл,Ярославская,Углич,57.522387,38.301979,Центральный,34505,0
1111,обл,Ярославская,Ярославль,57.621614,39.897878,Центральный,591486,1


## Список только столиц субъектов:

In [7]:
df_cities_centres

,reg_type,reg,city,lat,long,fed_reg,population,is_centre
0,Респ,Адыгея,Майкоп,44.609827,40.100653,Южный,144055,1
1,Респ,Алтай,Горно-Алтайск,51.958268,85.960296,Сибирский,62861,1
2,край,Алтайский,Барнаул,53.348115,83.779836,Сибирский,635585,1
3,обл,Амурская,Благовещенск,50.290593,127.527219,Дальневосточный,214397,1
4,обл,Архангельская,Архангельск,64.539299,40.517008,Северо-Западный,348716,1
...,...,...,...,...,...,...,...,...
80,Чувашия,Чувашская Республика,Чебоксары,56.143938,47.248872,Приволжский,447929,1
81,АО,Чукотский,Анадырь,64.731435,177.501575,Дальневосточный,13053,1
82,Респ,Саха /Якутия/,Якутск,62.028027,129.732572,Дальневосточный,269486,1
83,АО,Ямало-Ненецкий,Салехард,66.549357,66.608399,Уральский,42494,1


## Дальше матрицы расстояний:

In [8]:
# Все города
df_locations = df_cities[['lat', 'long']]
cities_names = df_cities['city']
df_locations.index = cities_names
dist_matrix = pd.DataFrame(distance_matrix(df_locations.values, df_locations.values), index=df_locations.index, columns=df_locations.index)

# Центры регионов
df_locations_centres = df_cities_centres[['lat', 'long']]
cities_names_centres = df_cities_centres['city']
df_locations_centres.index = cities_names_centres
dist_matrix_centres = pd.DataFrame(distance_matrix(df_locations_centres.values, df_locations_centres.values), index=df_locations_centres.index, columns=df_locations_centres.index)

## Матрица расстояний по всем городам:

In [9]:
dist_matrix

city,Адыгейск,Майкоп,Горно-Алтайск,Алейск,Барнаул,Белокуриха,Бийск,Горняк,Заринск,Змеиногорск,...,Любим,Мышкин,Переславль-Залесский,Пошехонье,Ростов,Рыбинск,Тутаев,Углич,Ярославль,Адлер
city,,,,,,,,,,,,,,,,,,,,,
Адыгейск,0.000000,0.949258,47.302953,44.249190,45.386944,46.343707,46.656972,42.714764,46.585401,43.453222,...,13.566438,12.930752,11.862428,13.628132,12.329134,13.174189,12.993677,12.675173,12.762879,1.610603
Майкоп,0.949258,0.000000,46.444660,43.400539,44.544683,45.487078,45.804818,41.854024,45.744387,42.592961,...,13.764653,13.280782,12.190099,13.930989,12.612619,13.495854,13.269581,13.037232,13.013368,1.186882
Горно-Алтайск,47.302953,46.444660,0.000000,3.225364,2.585745,0.976992,0.945942,4.597415,2.028366,3.856979,...,45.723984,47.862397,47.347665,47.294782,46.817386,47.494041,46.798044,47.982023,46.409264,46.830688
Алейск,44.249190,43.400539,3.225364,0.000000,1.316670,2.259735,2.434930,1.990505,2.470985,1.459589,...,42.499771,44.640406,44.129653,44.070715,43.597066,44.271134,43.575405,44.760989,43.187246,43.813916
Барнаул,45.386944,44.544683,2.585745,1.316670,0.000000,1.810570,1.646380,3.299269,1.206065,2.707921,...,43.383581,45.542507,45.053094,44.955739,44.509423,45.166730,44.473232,45.669027,44.089557,44.974062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Рыбинск,13.174189,13.495854,47.494041,44.271134,45.166730,46.521076,46.681752,43.185388,46.277281,43.873357,...,1.855317,0.480596,1.312343,0.527929,1.023268,0.000000,0.702202,0.765640,1.123731,14.648100
Тутаев,12.993677,13.269581,46.798044,43.575405,44.473232,45.824906,45.986761,42.486517,45.584959,43.174932,...,1.251959,1.085439,1.321306,0.762302,0.669761,0.702202,0.000000,1.282143,0.436787,14.434113
Углич,12.675173,13.037232,47.982023,44.760989,45.669027,47.008027,47.175814,43.652667,46.785378,44.344356,...,2.528449,0.306325,0.960937,1.280211,1.179361,0.765640,1.282143,0.000000,1.598981,14.175852


## Матрица расстояний только по региональным центрам:

In [10]:
dist_matrix_centres

city,Майкоп,Горно-Алтайск,Барнаул,Благовещенск,Архангельск,Астрахань,Уфа,Белгород,Брянск,Улан-Удэ,...,Хабаровск,Абакан,Ханты-Мансийск,Челябинск,Грозный,Чебоксары,Анадырь,Якутск,Салехард,Ярославль
city,,,,,,,,,,,,,,,,,,,,,
Майкоп,0.000000,46.444660,44.544683,87.610933,19.933820,8.146790,18.814019,6.943275,10.363838,67.869014,...,95.037445,52.145518,33.240785,23.769929,5.744610,13.569552,138.866456,91.308678,34.409354,13.013368
Горно-Алтайск,46.444660,0.000000,2.585745,41.600363,47.152674,38.314670,30.130637,49.393214,51.610992,21.624216,...,49.223710,5.760263,19.204871,24.767377,41.178867,38.937054,92.428132,44.915612,24.236249,46.409264
Барнаул,44.544683,2.585745,0.000000,43.854098,44.686854,36.399841,27.856510,47.274088,49.414679,23.852456,...,51.512035,7.673075,16.628007,22.452308,39.380485,36.637795,94.410510,46.765316,21.659433,44.089557
Благовещенск,87.610933,41.600363,43.854098,0.000000,88.169169,79.568088,71.707209,90.941914,93.208686,20.002658,...,7.750776,36.246264,59.481370,66.305504,82.125642,80.491457,52.018979,11.942819,63.051169,87.935460
Архангельск,19.933820,47.152674,44.686854,88.169169,0.000000,19.675375,18.290631,14.485213,12.863618,68.260095,...,95.899658,52.062866,28.719939,22.893155,21.844691,10.761044,136.984702,89.250900,26.168703,6.945335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Чебоксары,13.569552,38.937054,36.637795,80.491457,10.761044,9.811611,8.822249,12.019191,13.206376,60.489060,...,88.146143,44.261200,22.305031,14.186052,12.919423,0.000000,130.535481,82.693308,21.978718,7.498042
Анадырь,138.866456,92.428132,94.410510,52.018979,136.984702,130.741968,121.954111,141.622775,143.596686,71.097148,...,45.452182,86.759114,108.547314,116.494629,133.531614,130.535481,0.000000,47.845440,110.908076,137.787253
Якутск,91.308678,44.915612,46.765316,11.942819,89.250900,83.164746,74.134323,93.845449,95.771165,24.382005,...,14.571938,39.179276,60.722901,68.676035,86.092152,82.693308,47.845440,0.000000,63.285887,89.942697


# Тутушный датасет:

In [11]:
tutu = pd.read_csv('result_export.created_2020-03-17_13-46.csv')
# Сразу заменяем названия городов которые не бьются
tutu = tutu.replace(['Симферополь (Крым)', 'Ростов Великий', 'Новгород Великий', 'Зеленогорск (ЗАТО)', 'Вышний Волочёк', 'Фокино (ЗАТО)', 'Пикалёво', 'Большой Камень (ЗАТО)', 'Семёнов', 'Пугачёв', 'Озёры', 'Артёмовский', 'Олёкминск', 'Очёр', 'Озёрск', 'Трёхгорный', 'Чёрмоз', 'Артёмовск', 'Гусиноозёрск', 'Ербогачён', 'Артём', 'Заозёрный', 'Плёс', 'Жигулёвск', 'Новохопёрск', 'Будённовск', 'Киселёвск'], ['Симферополь', 'Ростов', 'Великий Новгород', 'Зеленогорск', 'Вышний Волочек', 'Фокино', 'Пикалево', 'Большой Камень', 'Семенов', 'Пугачев', 'Озеры', 'Артемовский', 'Олекминск', 'Очер', 'Озерск', 'Трехгорный', 'Чермоз', 'Артемовск', 'Гусиноозерск', 'Ербогачен', 'Артем', 'Заозерный', 'Плес', 'Жигулевск', 'Новохоперск', 'Буденновск', 'Киселевск'])

In [12]:
tutu_cities = set(list(tutu.arrival.values) + list(tutu.departure.values))

## Города которые есть в датасете туту но нет в df_cities:

In [13]:
lst = []
for c in tutu_cities:
    if c not in set(df_cities.city):
        lst.append(c)
print(lst, '\n\n', len(lst))

['Багдарин', 'Мама', 'Артышта', 'Сеймчан', 'Казачинское', 'Пено', 'Южно-Курильск', 'Ожерелье', 'Крутинка', 'Саккырыр', 'Кепервеем', 'Ноглики', 'Омсукчан', 'Сонково', 'Усть-Куйга', 'Новоабзаково', 'Таксимо', 'Иловля', 'Северо-Енисейский', 'Хоста', 'Черский', 'Симеиз', 'Берёзовый', 'Кослан', 'Железнодорожный', 'Пряжа', 'Староминская', 'Эвенск', 'Игрим', 'Приобье', 'Жиганск', 'Хотынец', 'Каргасок', 'Кугеси', 'Талакан', 'Пржевальское', 'Салми', 'Изборск', 'Охотск', 'Депутатский', 'Чокурдах', 'Толька', 'Зырянка', 'Лоо', 'Усть-Нера', 'Славянка', 'Дмитриев-Льговский', 'Павелец', 'Богородское', 'Карагайский', 'Берёзово', 'Псебай', 'Нижнеангарск', 'Ербогачен', 'Сычёвка', 'Байкит', 'Чернолучье', 'Батагай', 'Маркс ', 'Хандыга', 'Белая Гора', 'Залахтовье', 'Бежаницы', 'Светлогорск (Красноярск.)', 'Забайкальск', 'Усть-Кокса', 'Хонуу', 'Ванавара', 'Озеро-Карачинское', 'Оленёк', 'Туруханск', 'Сунтар', 'Чумикан', 'Чара', 'Диксон', 'Саскылах', 'Тикси', 'Усть-Качка', 'Максатиха', 'Ванино', 'Хатанга', 'М

## Сортировка записей с не попавшими в наш датасет городами по количеству пассажиров

In [14]:
tutu[tutu.departure.isin(lst) | tutu.arrival.isin(lst)].sort_values(by='passengers', ascending=False)

,transport,departure,arrival,passengers
3352,bus,Красноярск,Мотыгино,17792
3409,bus,Тверь,Максатиха,16303
3486,bus,Санкт-Петербург,Бежаницы,12840
3530,bus,Санкт-Петербург,Салми,12006
3651,bus,Санкт-Петербург,Сандово,9236
...,...,...,...,...
16838,train,Мичуринск,Хоста,13
14273,train,Таксимо,Нерюнгри,13
16881,train,Хоста,Россошь,12
14121,train,Слюдянка,Забайкальск,12
